<h1>
<hr style=" border:none; height:3px;">
<center>SHIA - Results analysis</center>
<hr style=" border:none; height:3px;">
</h1>

<center><img src='https://netacad.centralesupelec.fr/img/cs.jpg' width=200></center>

<h4><center>Louis LHOTTE | Ambroise MARTIN-ROUVILLE | Paul-Alexandre MARENGHI | Paul LE BELLOCH</center></h4>

# 0 - Imports

In [18]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import ttest_rel

# Data analysis

# I.1 - Reading

In [7]:
df_joy = pd.read_excel('./Data/results.xlsx', sheet_name='Joy')
df_joy.head(12)

,Emotion,# questions,N° example,Music pertinence,VAD pertinence
0,Joy,1.0,1.0,2.0,NaN
1,NaN,NaN,2.0,4.0,NaN
2,NaN,NaN,3.0,1.0,NaN
3,NaN,NaN,4.0,3.0,NaN
4,NaN,NaN,5.0,1.0,NaN
5,NaN,NaN,6.0,0.0,NaN
6,NaN,NaN,7.0,3.0,NaN
7,NaN,NaN,8.0,4.0,NaN
8,NaN,NaN,9.0,1.0,NaN
9,NaN,NaN,10.0,1.0,NaN


In [4]:
df_anger = pd.read_excel('./Data/results.xlsx', sheet_name='Anger')
df_anger.head(1)

,Emotion,# questions,N° example,Range
0,Anger,1.0,1.0,NaN


In [5]:
df_sadness = pd.read_excel('./Data/results.xlsx', sheet_name='Sadness')
df_sadness.head(1)

,Emotion,# questions,N° example,Music pertinence,VAD pertinence
0,Sadness,1.0,1.0,4.0,NaN


In [6]:
df_fear = pd.read_excel('./Data/results.xlsx', sheet_name='Fear')
df_fear.head(1)

,Emotion,# questions,N° example,Music pertinence,VAD pertinence
0,Fear,1.0,1.0,5.0,NaN


## I.2 - Pre-processing

In [15]:
def clean_emotion_dataframe(df_raw):
    df_cleaned = df_raw.ffill()
    df_cleaned = df_cleaned.dropna(subset=["N° example"])
    df_cleaned = df_cleaned.rename(columns={
        "Emotion": "Emotion",
        "# questions": "# question",
        "N° example": "# example",
        "Music pertinence": "Music pertinence",
        "VAD pertinence": "AD pertinence"
    })
    df_cleaned = df_cleaned.reset_index(drop=True)
    df_cleaned = df_cleaned[['Emotion', '# question', 'Music pertinence', 'AD pertinence']]
    return df_cleaned

cleaned_fear_df = clean_emotion_dataframe(df_fear)
cleaned_fear_df.head()

,Emotion,# question,Music pertinence,AD pertinence
0,Fear,1.0,5.0,NaN
1,Fear,1.0,4.0,NaN
2,Fear,1.0,3.0,NaN
3,Fear,1.0,4.0,NaN
4,Fear,1.0,4.0,NaN


In [17]:
cleaned_joy_df = clean_emotion_dataframe(df_joy)
cleaned_sadness_df = clean_emotion_dataframe(df_sadness)
# cleaned_anger_df = clean_emotion_dataframe(df_anger) # → Need harmonization

## I.3 - Paired T-test

<div class="alert alert-block alert-info">

The <b>t-test</b> approach involves conducting <b>paired t-tests</b> within each emotion category (e.g., joy, sadness, anger) to compare sentiment scores at different stages — specifically testing whether <b>μ(score₁) &lt; μ(score₂)</b> and/or <b>μ(score₁) &lt; μ(score₃)</b>, where score₁ (resp. score₂) is the score of the experiment after one question (resp. 2 questions), . This test assumes normally distributed differences and assesses whether there is a statistically significant increase in sentiment score over time within each emotion group. We expect that for some emotions, especially those reflecting improvement or engagement (e.g., sadness), the scores might increase, supporting the hypothesis that <b>sentiments evolve positively</b> across the interaction.

</div>


In [ ]:
df_all = pd.concat([cleaned_fear_df, cleaned_joy_df, cleaned_sadness_df])

# Pivot for paired t-test: one row per example, columns are score after 1, 4, and 7 questions
def pivot_for_ttest(df, value_col):
    return df.pivot(index=df.groupby('# question').cumcount(), columns="# question", values=value_col)

# Conduct paired t-tests for each emotion
ttest_results = {}
for emotion, group in df_all.groupby("Emotion"):
    pivoted = group.pivot(index=group.groupby('# question').cumcount(), columns="# question", values="Music pertinence")
    if {1, 4, 7}.issubset(pivoted.columns):
        t1_4 = ttest_rel(pivoted[1], pivoted[4])
        t1_7 = ttest_rel(pivoted[1], pivoted[7])
        t4_7 = ttest_rel(pivoted[4], pivoted[7])
        ttest_results[emotion] = {
            "1 vs 4": t1_4,
            "1 vs 7": t1_7,
            "4 vs 7": t4_7
        }

# Visualizations
sns.set(style="whitegrid")

plt.figure(figsize=(10, 6))
sns.boxplot(data=df_all, x="# question", y="Music pertinence", hue="Emotion")
plt.title("Music Pertinence by Emotion and Number of Questions")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
sns.lineplot(data=df_all, x="# question", y="Music pertinence", hue="Emotion", marker="o")
plt.title("Music Pertinence Trends Across Questions")
plt.tight_layout()
plt.show()

## I.4 - Friedman Test

<div class="alert alert-block alert-info">
The <b>Friedman test</b>, a non-parametric alternative to repeated-measures ANOVA, is used to compare scores after 1, 4, and 7 questions within each emotion category. It tests whether there are <b>statistically significant differences in the distribution of scores as the number of questions increases</b>, without assuming normality — appropriate given the small sample size (n=10). We expect the Friedman test to reveal <b>systematic changes in sentiment over the course of the interaction</b>, indicating that emotional responses evolve with the number of questions asked.
</div>